In [ ]:
import requests
import pandas as pd

In [ ]:
url = "http://b2find.eudat.eu/solr/select"
# url = "http://localhost:5000/solr/select"


In [ ]:
import time
years = year = 60 * 60 * 24 * 365
year1epochsec = 62135600400
now = year1epochsec + int(time.time())

facet = "extras_TempCoverageBegin"
# facet = "extras_TempCoverageEnd"
min_range = now-50*years
max_range = now+30*years
interval = 1*year
interval

In [ ]:
solr_params = {
    'echoParams': 'none',
    'rows': 0,
    'wt': 'json',
    'q': '*',
    'fq': f'{{!frange l=0}} sub(extras_TempCoverageEnd,extras_TempCoverageBegin)',
    'facet': 'true',
    'facet.sort': 'index',
    'facet.limit': -1,
    'indent': 'false',
    'facet.range':[
        facet, 
    ],
    'facet.range.start': min_range,
    'facet.range.end': max_range,
    'facet.range.gap': interval,
}

In [ ]:
import time
run_start = time.time()

resp = requests.post(url, data=solr_params)
run_end = time.time() - run_start
print(resp.ok, run_end)

In [ ]:
result = resp.json()
result_begin = result["facet_counts"]["facet_ranges"][facet]["counts"]
x_begin = result_begin[0::2]
y = result_begin[1::2]
x = [int(i)-year1epochsec for i in x_begin]


In [ ]:
df = pd.DataFrame(list(zip(x, y)),
               columns =['start', 'counts'])
df

In [ ]:
df["start"]=pd.to_datetime(df["start"],unit="s")
df

In [ ]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure

source = ColumnDataSource(df)

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(df.start.loc[0], df.start.loc[50]))

p.line('start', 'counts', source=source)
p.yaxis.axis_label = 'datasets'

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('start', 'counts', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))